# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.num_classes = num_classes
        self.feature = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, num_classes)
        )
    def forward(self,x):
        x = self.feature(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = F.log_softmax(x,dim = 1)
        return x
    
model = SimpleNet()

def train(model,epoch, optimizer, train_loader, device, log_interval=100):
    model.train()
    for batch_idx, (id_,feature, label) in enumerate(train_loader):
        feature, label = feature.to(device, dtype=torch.float), label.to(device, dtype=torch.long)
        optimizer.zero_grad()
        output = model(feature)
        loss = nn.CrossEntropyLoss()(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(feature), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def eval_(model, epoch, eval_loader, device, best_acc, best_weights):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (id_, feature, label) in enumerate(eval_loader):
            feature, label = feature.to(device, dtype=torch.float), label.to(device, dtype=torch.long)
            output = model(feature)
            test_loss += nn.CrossEntropyLoss()(output, label).item()
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(label.view_as(pred)).sum().item()
    test_loss /= (batch_idx+1)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(eval_loader.dataset),
        100. * correct / len(eval_loader.dataset)))
    return best_acc, best_weights
def train_eval(model, batch_size, epoch_num, use_cuda, lr, train_set, eval_set):
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.to(device)
    
    print("initialize weights done")
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
 
    print("begin train....")
    best_acc,best_weights = 0, "weights/sne_0_0.pt"
    for epoch in range(epoch_num):
        train(model, epoch, optimizer, train_loader, device)
        eval_(model, epoch, test_loader,device)
        
    torch.save(model.state_dict(),"my_ic_final_%s.pt"%epoch_num)
    return model, optimizer, train_loader, eval_loader, best_weights
    
def goon_train(batch_size, start_epoch, end_epoch, use_cuda,  model, optimizer, train_loader, eval_loader):
    print("begin train....")
    device = torch.device("cuda" if use_cuda else "cpu")
    best_acc,best_weights = 0, "weights/sne_0_0.pt"
    for epoch in range(start_epoch, end_epoch):
        train(model, epoch, optimizer, train_loader, device)
        best_acc,best_weights =eval_(model, epoch, eval_loader,device, best_acc, best_weights)
        
    torch.save(model.state_dict(),"my_ic_final_%s.pt"%end_epoch)
    return model, optimizer, train_loader, eval_loader, best_weights

model, optimizer, train_loader, eval_loader, best_weights = train_eval(model,128, 100, True, 0.1, train_set, eval_set)
for param_group in optimizer.param_groups:
    param_group['lr'] = 0.01
model, optimizer, train_loader, eval_loader, best_weights= goon_train(128, 100, 200, True, model, optimizer, train_loader, eval_loader)
for param_group in optimizer.param_groups:
    param_group['lr'] = 0.001
model, optimizer, train_loader, eval_loader, best_weights= goon_train(128, 200, 300, True, model, optimizer, train_loader, eval_loader)


# TODO:define loss function and optimiter
criterion = 
optimizer = 

Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        
        
        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
    
    
    


#### Q5:
Please print the training and testing accuracy.